In [20]:
import cv2
import torch
import torch.nn as nn
import numpy as np
import tkinter as tk
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image, ImageTk

# 그림을 그리기 위한 캔버스 크기 설정
canvas_width = 300
canvas_height = 300

# 그리기 상태 변수 초기화
drawing = False
last_x, last_y = None, None

# OpenCV에서 사용할 빈 이미지 생성
image = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255

def load_image():
    global img
    # 이미지를 RGB로 변환
    img_array = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img_array)
    img = ImageTk.PhotoImage(img)
    canvas.create_image(0, 0, anchor=tk.NW, image=img)
    canvas.image = img  # 이미지 객체의 참조를 유지

def clear_button():
    global image
    image = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255
    load_image()

def start_drawing(event):
    global drawing, last_x, last_y
    drawing = True
    last_x, last_y = event.x, event.y

def draw(event):
    global last_x, last_y, image
    if drawing:
        cv2.line(image, (last_x, last_y), (event.x, event.y), (0, 0, 0), 7)
        last_x, last_y = event.x, event.y
        load_image()

def evaluate_image():
    global image
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # 예측 수행
    input_image = torch.tensor(np.array(gray_image) / 255, dtype=torch.float32)
    input_image = input_image.reshape(1, 1, 300, 300)
    dataset = TensorDataset(input_image)
    eval_image = DataLoader(dataset)
    with torch.no_grad():
        output = model(input_image)
        prediction = torch.argmax(output, dim=1)
    if (np.array(prediction) == [0]):
        print('O')
    elif (np.array(prediction) == [1]):
        print("X")

def stop_drawing(event):
    global drawing
    drawing = False

# Tkinter 윈도우 생성
root = tk.Tk()
root.title("OX_Classification")

# 캔버스 생성
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height)
canvas.pack()

# 이벤트 바인딩
canvas.bind("<ButtonPress-1>", start_drawing)
canvas.bind("<B1-Motion>", draw)
canvas.bind("<ButtonRelease-1>", stop_drawing)

# 버튼 생성
button = tk.Button(root, text="clear", command=clear_button)
button.pack(pady=10)

evaluate_button = tk.Button(root, text="평가", command=evaluate_image)
evaluate_button.pack(pady=5)

model = torch.jit.load('model_state_dict.pt')
model.eval()

# 초기 이미지 로드
load_image()

# Tkinter 이벤트 루프 시작
root.mainloop()

O
X


# 회고록

처음 모델을 만들었을 때 모든 예측이 O 혹은 X 하나로 통일됐다.

모델이 문제일까 전처리가 문제일까 고민을 했는데, 모델이 문제면 예측률이 떨어지지 모든 예측이 하나로 통일되지 않을 것이라고 생각했다.

데이터 전처리에 대해 문제가 뭘까 고민했다.

label data를 하나의 열로 처리를 하게 되면 "O"와 "O가 아니다"로 구분되는 것이고, 두 개의 열로 처리를 시작해서 O일 때 [1, 0] / X일 때 [0, 1]으로 구분하기로 했다.

이렇게 구분하고 모델에 넣었는데 이 형태는 원하지 않고 다시 하나의 열로 변경해서 모델에 넣었다.

이렇게 하니까 예측률이 상승했다.

그리고 추가적으로 알게 된 것이 pytorch의 CrossEntropyLoss 함수는 함수 내에서 label_smoothing을 실시한다. + softmax가 적용된다.

Dataset과 DataLoader를 왜 사용하는지 정확하게 이해하지 못했는데 이번을 계기로 이해할 수 있게 됐다.

Dataset은 자신만의 Dataset을 만들 수 있도록 해주고, DataLoader는 배치 크기를 조절하거나 데이터의 병렬처리를 통해 데이터를 로드할 수 있다.

이전에 처음 pytorch를 사용하고, 인공지능 모델을 만들 때는 작성하고 돌아가기만 하면 만족하고 끝냈는데, 다시 공부하면서 인공지능 모델에 사용되는 손실함수, 옵티마이저, learning_rate 등의 의미를 알게 되는 계기가 됐다.